# Create an InterActive Graph of NTRL Lab Analysis Results

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px

In [2]:
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------

In [3]:
class LabResults():
    def __init__(self, monthyear, filePath):
        #____monthyear format examle is datetime.date(2020,7,1)
        #____filepath example is '\\thgobna001\userdata\THPAL\...\ANALYSIS RESULTS 2020\7) July _2020.xlsx'
        #____add r' at the start of the filepath to apply the regex that allows the use of the filepath as raw
        #____sef.workbook is the uploaded excel workbook of the entire analysis files so it takes time to load
        #____Create an instance of the workbook by giving it a varible name with the sample format: Jan_2020
        
        import openpyxl
        import io
        with open(filePath, "rb") as f:
            in_mem_file = io.BytesIO(f.read())
        
        self.workbook = openpyxl.load_workbook(in_mem_file, data_only=True)
        self.monthyear = monthyear   
        
    def generate_df_from_analysis(self, tabname, list_lab_index):
        #____tabname is a string type name of the tab in the laboratory analysis excel file.
        #____-------Example tabnames: '104PU01', '105TK03', '106TK01'
        #____-------Inside these tabs are the specific analysis results done on the sample taken from the 'tabname' area .
        #____list_lab_index is a list with the sample format is: ['B', 'C', 'E', '104_Pb']
        #____-------the first letter 'B' is the column letter where the day of the month is written as integers 1,2,3..30
        #____-------the second letter 'C' is the column letter where the sampling time is written; e.g. '7:00:00 AM'
        #____-------the third letter 'E' is the column where the results of the specific analysis are listed
        #____-------the last item in the list is the string you'll use as the new title; e.g. '104_Pb'
        
        workbookTab = self.workbook[tabname]
        self.list_lab_index = list_lab_index
        col_sampleDate = self.list_lab_index[0]
        col_sampleTime = self.list_lab_index[1]
        col_analysis = self.list_lab_index[2]
        col_title = self.list_lab_index[3]
        currentmonthyr = self.monthyear
        
        #get the start and end sample dates
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
        #gets the date day from the B column of the analysis file if there is value in the cell
        for i in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
                Smpl_day_index.append(i)
        #get the ending row that has date day
        for j in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
                smpl_end_index = j
        for y in range(0, len(Smpl_day_index)):
            if Smpl_day_index[y] != Smpl_day_index[-1]:      
                for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                    if workbookTab[str(col_analysis)+str(x)].value is None: 
                        pass
                    else:
                        if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                            date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))  
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)
                        else:
                            date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))  
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)

            if Smpl_day_index[y] == Smpl_day_index[-1]:
                for x in range(Smpl_day_index[y], smpl_end_index+1):
                    if workbookTab[str(col_analysis)+str(x)].value is None:   
                        pass
                    else:
                        if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                            date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)
                        else:
                            date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                            +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                            Chem_Element_datelist.append(parse(date))
                            chem_element = workbookTab[str(col_analysis)+str(x)].value
                            Chem_Element_list.append(chem_element)

        Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
        Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
        Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
        Chem_Element_df.set_index('DATETIME', inplace=True)

        return Chem_Element_df

In [4]:
#monthyear
currentmonthyr_01 = datetime.date(2020,1,1)
currentmonthyr_02 = datetime.date(2020,2,1)
currentmonthyr_03 = datetime.date(2020,3,1)
currentmonthyr_04 = datetime.date(2020,4,1)
currentmonthyr_05 = datetime.date(2020,5,1)
currentmonthyr_06 = datetime.date(2020,6,1)
currentmonthyr_07 = datetime.date(2020,7,1)

In [5]:
#filepath
filepath_01 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\JANUARY 2020\1) JANUARY_2020.xlsx'
filepath_02 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\FEBRUARY 2020\2) FEBRUARY _2020.xlsx'
filepath_03 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\MARCH 2020\3) March _2020.xlsx'
filepath_04 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\APRIL 2020\4) April _2020.xlsx'
filepath_05 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\MAY 2020\5) MAY_2020.xlsx'
filepath_06 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\JUNE 2020\6) June _2020.xlsx'
filepath_07 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\7) July _2020.xlsx'

In [6]:
#list_lab_index for ntrl
ntrl_Pb = ['B', 'C', 'E', '104_Pb']
ntr_Zn =  ['B', 'C', 'F', '104_Zn']
ntrl_Cu = ['B', 'C', 'G', '104_Cu']
ntrl_Ni = ['B', 'C', 'H', '104_Ni']
ntrl_Co = ['B', 'C', 'I', '104_Co']
ntrl_Mn = ['B', 'C', 'K', '104_Mn']
ntrl_Cr = ['B', 'C', 'L', '104_Cr']
ntrl_Ca = ['B', 'C', 'M', '104_Ca']
ntrl_Si = ['B', 'C', 'N', '104_Si']
ntrl_Al = ['B', 'C', 'O', '104_Al']
ntrl_Mg = ['B', 'C', 'P', '104_Mg']
ntrl_Fe2 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH = ['B', 'C', 'BB', '104_pH60']
ntrl_ORP = ['B', 'C', 'AJ', '104_ORP']
ntrl_NTU = ['B', 'C', 'AN', '104_NTU']
ntrl_FT = ['B', 'C', 'AW', '104_FT']

In [20]:
Jan_2020 = LabResults(currentmonthyr_01, filepath_01)
print('one down')

one down


In [24]:
#NTRL January 2020
Jan2020_ntrl_Pb_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_Pb)
Jan2020_ntrl_Zn_df = Jan_2020.generate_df_from_analysis('104PU01', ntr_Zn)
Jan2020_ntrl_Cu_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_Cu)
Jan2020_ntrl_Ni_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_Ni)
Jan2020_ntrl_Co_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_Co)
Jan2020_ntrl_Mn_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_Mn)
Jan2020_ntrl_Cr_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_Cr)
Jan2020_ntrl_Ca_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_Ca)
Jan2020_ntrl_Si_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_Si)
Jan2020_ntrl_Al_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_Al)
Jan2020_ntrl_Mg_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_Mg)
Jan2020_ntrl_Fe2_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_Fe2)
Jan2020_ntrl_pH_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_pH)
Jan2020_ntrl_ORP_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_ORP)
Jan2020_ntrl_NTU_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_NTU)
Jan2020_ntrl_FT_df = Jan_2020.generate_df_from_analysis('104PU01', ntrl_FT)

In [ ]:
Jan2020_ntrl_FT_df = January_2020.generate_df_from_analysis('105TK05', ntrl_FT)

In [28]:
#list_lab_index for 105TK05
deznTK05_Pb = ['B', 'C', 'E', '105TK05_Pb']
deznTK05_Zn =  ['B', 'C', 'F', '105TK05_Zn']
deznTK05_Cu = ['B', 'C', 'G', '105TK05_Cu']
deznTK05_Ni = ['B', 'C', 'H', '105TK05_Ni']
deznTK05_Co = ['B', 'C', 'I', '105TK05_Co']
deznTK05_Fe2 = ['B', 'C', 'J', '105TK05_Fe']
deznTK05_Mn = ['B', 'C', 'K', '105TK05_Mn']
deznTK05_Cr = ['B', 'C', 'L', '105TK05_Cr']
deznTK05_Ca = ['B', 'C', 'M', '105TK05_Ca']
deznTK05_Si = ['B', 'C', 'N', '105TK05_Si']
deznTK05_Al = ['B', 'C', 'O', '105TK05_Al']
deznTK05_Mg = ['B', 'C', 'P', '105TK05_Mg']
deznTK05_pH = ['B', 'C', 'AF', '105TK05_pH']
deznTK05_ORP = ['B', 'C', 'AG', '105TK05_ORP']
deznTK05_PS90 = ['B', 'C', 'AP', '105TK05_PS90']

In [29]:
#105TK05 January 2020
Jan2020_df_deznTK05_Pb = January_2020.generate_df_from_analysis('105TK05', deznTK05_Pb)
Jan2020_df_deznTK05_Zn = January_2020.generate_df_from_analysis('105TK05', deznTK05_Zn)
Jan2020_df_deznTK05_Cu = January_2020.generate_df_from_analysis('105TK05', deznTK05_Cu)
Jan2020_df_deznTK05_Ni = January_2020.generate_df_from_analysis('105TK05', deznTK05_Ni)
Jan2020_df_deznTK05_Co = January_2020.generate_df_from_analysis('105TK05', deznTK05_Co)
Jan2020_df_deznTK05_Fe2 = January_2020.generate_df_from_analysis('105TK05', deznTK05_Fe2)
Jan2020_df_deznTK05_Mn = January_2020.generate_df_from_analysis('105TK05', deznTK05_Mn)
Jan2020_df_deznTK05_Cr = January_2020.generate_df_from_analysis('105TK05', deznTK05_Cr)
Jan2020_df_deznTK05_Ca = January_2020.generate_df_from_analysis('105TK05', deznTK05_Ca)
Jan2020_df_deznTK05_Si = January_2020.generate_df_from_analysis('105TK05', deznTK05_Si)
Jan2020_df_deznTK05_Al = January_2020.generate_df_from_analysis('105TK05', deznTK05_Al)
Jan2020_df_deznTK05_Mg = January_2020.generate_df_from_analysis('105TK05', deznTK05_Mg)
Jan2020_df_deznTK05_pH = January_2020.generate_df_from_analysis('105TK05', deznTK05_pH)
Jan2020_df_deznTK05_ORP = January_2020.generate_df_from_analysis('105TK05', deznTK05_ORP)
Jan2020_df_deznTK05_PS90 = January_2020.generate_df_from_analysis('105TK05', deznTK05_PS90)

In [35]:
#list_lab_index for 105TK03
deznTK03_Zn = ['B', 'C', 'D', '105TK03_Zn']
deznTK03_Zn_change =  ['B', 'C', 'BB', '105TK03_Zn_Change_Ratio']
deznTK03_TK05_Zn = ['B', 'C', 'BD', '105TK05_Zn']
deznTK03_TK01_Zn = ['B', 'C', 'BE', '106TK01_Zn']
deznTK03_Zn_Ni = ['B', 'C', 'BF', '106TK01_Zn/Ni']
deznTK03_Zn_Droprate = ['B', 'C', 'BG', '106TK01_Zn_DropRate']
deznTK03_VE04Zn_prcnt = ['B', 'C', 'BH', '106VE04_%Zn']
deznTK03_TH01Zn_prcnt = ['B', 'C', 'BI', '106TH01_%Zn']
deznTK03_NTRL_Ni = ['B', 'C', 'BJ', '104PU01_Ni'] #not true for january 2020 resulst
deznTK03_TK05_Ni = ['B', 'C', 'BK', '105TK05_Ni'] #not true for january 2020 results
deznTK03_Ni_Loss_TK05 = ['B', 'C', 'BL', 'Ni_Loss_TK05'] #not true for january 2020 results


In [36]:
#105TK03 January 2020
Jan2020_df_deznTK03_Zn = January_2020.generate_df_from_analysis('105TK03', deznTK03_Zn)
Jan2020_df_deznTK03_Zn_change = January_2020.generate_df_from_analysis('105TK03', deznTK03_Zn_change)
Jan2020_df_deznTK03_TK05_Zn = January_2020.generate_df_from_analysis('105TK03', deznTK03_TK05_Zn)
Jan2020_df_deznTK03_TK01_Zn = January_2020.generate_df_from_analysis('105TK03', deznTK03_TK01_Zn)
Jan2020_df_deznTK03_Zn_Ni = January_2020.generate_df_from_analysis('105TK03', deznTK03_Zn_Ni)
Jan2020_df_deznTK03_Zn_Droprate = January_2020.generate_df_from_analysis('105TK03', deznTK03_Zn_Droprate)
Jan2020_df_deznTK03_VE04Zn_prcnt = January_2020.generate_df_from_analysis('105TK03', deznTK03_VE04Zn_prcnt)
Jan2020_df_deznTK03_TH01Zn_prcnt = January_2020.generate_df_from_analysis('105TK03', deznTK03_TH01Zn_prcnt)

#Jan2020_df_deznTK03_NTRL_Ni = January_2020.generate_df_from_analysis('105TK03', deznTK03_NTRL_Ni)
#Jan2020_df_deznTK03_TK05_Ni = January_2020.generate_df_from_analysis('105TK03', deznTK03_TK05_Ni)
#Jan2020_df_deznTK03_Ni_Loss_TK05 = January_2020.generate_df_from_analysis('105TK03', deznTK03_Ni_Loss_TK05)

In [ ]:
#-----------------------------------------------
#105TK03
DeZn_TK03 = wb1['105TK03']
DeZn_TK03_105TK03_Zn_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'D', '105TK03_Zn')
DeZn_TK03_105TK03_Zn_Change_Ratio_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BB', '105TK03_Zn_Change_Ratio')
DeZn_TK03_105TK05_Zn_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BD', '105TK05_Zn')
DeZn_TK03_106TK01_Zn_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BE', '106TK01_Zn')
DeZn_TK03_106TK01_ZnNi_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BF', '106TK01_Zn/Ni')
DeZn_TK03_106TK01_Zn_DropRate_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BG', '106TK01_Zn_DropRate')
DeZn_TK03_106VE04_Znprcnt_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BH', '106VE04_%Zn')
DeZn_TK03_106TH01_Znprcnt_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BI', '106TH01_%Zn')
DeZn_TK03_104PU01_Ni_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BJ', '104PU01_Ni')
DeZn_TK03_105TK05_Ni_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BK', '105TK05_Ni')
DeZn_TK03_Ni_Loss_TK05_df = generate_df_from_analysis(DeZn_TK03, 'B', 'C', 'BL', 'Ni_Loss_TK05')

for i in range(0,len(DeZn_TK03_104PU01_Ni_df)):
    if DeZn_TK03_104PU01_Ni_df['104PU01_Ni'][i] == 0:
        DeZn_TK03_104PU01_Ni_df['104PU01_Ni'][i] = np.nan
        
#105TK05 Ni
for i in range(0,len(DeZn_TK03_105TK05_Ni_df)):
    if DeZn_TK03_105TK05_Ni_df['105TK05_Ni'][i] == 0:
        DeZn_TK03_105TK05_Ni_df['105TK05_Ni'][i] = np.nan

#105TK05 Ni Loss
for i in range(0,len(DeZn_TK03_Ni_Loss_TK05_df)):
    if DeZn_TK03_Ni_Loss_TK05_df['Ni_Loss_TK05'][i] == 0:
        DeZn_TK03_Ni_Loss_TK05_df['Ni_Loss_TK05'][i] = np.nan         
        
#105TK05 Zn
for i in range(0,len(DeZn_TK03_105TK05_Zn_df)):
    if DeZn_TK03_105TK05_Zn_df['105TK05_Zn'][i] == 0:
        DeZn_TK03_105TK05_Zn_df['105TK05_Zn'][i] = np.nan    
        
#106TK01 Zn
for i in range(0,len(DeZn_TK03_106TK01_Zn_df)):
    if DeZn_TK03_106TK01_Zn_df['106TK01_Zn'][i] == 0:
        DeZn_TK03_106TK01_Zn_df['106TK01_Zn'][i] = np.nan      


In [ ]:
#-----------------------------------------------
DeZn_FT01 = wb1['105FT01']
FT01A_Zn_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'D', 'Zn')
FT01B_Zn_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'E', 'Zn')
FT01C_Zn_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'F', 'Zn')
FT01D_Zn_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'G', 'Zn')
FT01E_Zn_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'H', 'Zn')
FT01F_Zn_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'I', 'Zn')

FT01A_ntu_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'J', 'ntu')
FT01B_ntu_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'K', 'ntu')
FT01C_ntu_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'L', 'ntu')
FT01D_ntu_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'M', 'ntu')
FT01E_ntu_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'N', 'ntu')
FT01F_ntu_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'O', 'ntu')

FT01A_ZDR_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'J', 'Zn_dr')
FT01B_ZDR_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'K', 'Zn_dr')
FT01C_ZDR_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'L', 'Zn_dr')
FT01D_ZDR_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'M', 'Zn_dr')
FT01E_ZDR_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'N', 'Zn_dr')
FT01F_ZDR_df = generate_df_from_analysis(DeZn_FT01, 'B', 'C', 'O', 'Zn_dr')

A = FT01A_Zn_df.join(FT01A_ntu_df, how='outer').join(FT01A_ZDR_df, how='outer')
A['Filter'] = 'FT01A'
A.reset_index(inplace=True)

B = FT01B_Zn_df.join(FT01B_ntu_df, how='outer').join(FT01B_ZDR_df, how='outer')
B['Filter'] = 'FT01B'
B.reset_index(inplace=True)

C = FT01C_Zn_df.join(FT01C_ntu_df, how='outer').join(FT01C_ZDR_df, how='outer')
C['Filter'] = 'FT01C'
C.reset_index(inplace=True)

D = FT01A_Zn_df.join(FT01D_ntu_df, how='outer').join(FT01D_ZDR_df, how='outer')
D['Filter'] = 'FT01D'
D.reset_index(inplace=True)

E = FT01E_Zn_df.join(FT01E_ntu_df, how='outer').join(FT01E_ZDR_df, how='outer')
E['Filter'] = 'FT01E'
E.reset_index(inplace=True)

F = FT01F_Zn_df.join(FT01F_ntu_df, how='outer').join(FT01F_ZDR_df, how='outer')
F['Filter'] = 'FT01F'
F.reset_index(inplace=True)
AtoF = A.append(B, ignore_index =True).append(C, ignore_index = True).append(D, ignore_index=True).append(E, ignore_index=True).append(F, ignore_index=True)
AtoF.interpolate(method='linear', limit_direction='forward', axis=0)
AtoF.dropna(inplace=True)

#-----------------------------------------------


In [2]:

#fill the nan values with the average of surrounding values
for i in range(0, len(df_main.columns)):
    #print(i, df_main.columns[i])
    ave_filler(df_main, df_main.columns[i])
df_main = df_main.fillna(method='bfill')

df_main2 = df_main[['104_Pb', '104_Zn', '104_Cu', '104_Ni', '104_Co', '104_Mn',
       '104_Cr', '104_Ca', '104_Si', '104_Al', '104_Mg', '104_Fe2', '104_pH',
       '104_ORP',]]
df_main2 = df_main2.dropna(how='any')

zn_drop_lst = list()
datelist = list()
for j in range(0, len(df_main2)):
    zn_drop = 0
    for i in range(0, len(df_main2.iloc[j])):
        #print(df_main2.iloc[j][i], " * " ,coeff_Zn_drop['Zn_drop'][i])
        temp_product = df_main2.iloc[j][i] * coeff_Zn_drop['Zn_drop'][i]
        zn_drop = zn_drop + temp_product
    zn_drop_new = zn_drop + lm_Zndrop.intercept_
    zn_drop_lst.append(zn_drop_new)
    datelist.append(df_main2.index[j])
zn_drop_and_date_lst = list(zip(datelist, zn_drop_lst))
df_zn_drop_and_date = pd.DataFrame(zn_drop_and_date_lst)
df_zn_drop_and_date = df_zn_drop_and_date.set_index(0)
df_zn_drop_and_date['Zn_drop_Pred'] = df_zn_drop_and_date[1]

# generates a dataframe for each column in the excel file
MS_106TK01 = generate_df_from_analysis(MS_lab, 'B', 'C', 'BD', 'Zn_drop_ACT')
pd.to_numeric(MS_106TK01["Zn_drop_ACT"], errors='coerce')
for i in range(1, len(MS_106TK01.Zn_drop_ACT)):
    try:
        if type(MS_106TK01.Zn_drop_ACT[i]) == str:
            print(i, MS_106TK01.Zn_drop_ACT[i])
            MS_106TK01.drop(MS_106TK01.index[i], inplace=True)
            print(i, MS_106TK01.Zn_drop_ACT[i])
    except:
        pass

print('Done ', datetime.datetime.now())
y = df_zn_drop_and_date['Zn_drop_Pred']
MS_y = MS_106TK01["Zn_drop_ACT"]*100

y_Pb_ntrl = df_main['104_Pb']
y_Zn_ntrl = df_main['104_Zn']
y_Cu_ntrl = df_main['104_Cu']
y_Ni_ntrl = df_main['104_Ni']
y_Co_ntrl = df_main['104_Co']
y_Mn_ntrl = df_main['104_Mn']
y_Cr_ntrl = df_main['104_Cr']
y_Ca_ntrl = df_main['104_Ca']
y_Si_ntrl = df_main['104_Si']
y_Al_ntrl = df_main['104_Al']
y_Mg_ntrl = df_main['104_Mg']
y_Fe2_ntrl = df_main['104_Fe2']
y_pH_ntrl = df_main['104_pH']
y_ORP_ntrl = df_main['104_ORP']


zn_drop_predictions = MS_106TK01.join(df_zn_drop_and_date, how ='outer')

Done 2020-08-01 10:03:01.322545
Done  2020-08-01 10:03:07.511968
VE04 PRED part Done 2020-08-01 10:03:30.436649


In [ ]:
####-------------------Space for the 106TK01 and 106VE04 data to plot-----------------------------------------
# generates a dataframe for each column in the Analysis excel file 104PU01
MS_TK01_Pb_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'E', '106TK01_Pb')
MS_TK01_Zn_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'F', '106TK01_Zn')
MS_TK01_Cu_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'G', '106TK01_Cu')
MS_TK01_Ni_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'H', '106TK01_Ni')
MS_TK01_Co_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'I', '106TK01_Co')
MS_TK01_Fe_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'J', '106TK01_Fe')
MS_TK01_Mn_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'K', '106TK01_Mn')
MS_TK01_Cr_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'L', '106TK01_Cr')
MS_TK01_Ca_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'M', '106TK01_Ca')
MS_TK01_Si_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'N', '106TK01_Si')
MS_TK01_Al_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'O', '106TK01_Al')
MS_TK01_Mg_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'P', '106TK01_Mg')
MS_TK01_pH_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'AH', '106TK01_pH')
MS_TK01_ORP_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'AI', '106_ORP')
MS_TK01_ZnNi_df = generate_df_from_analysis(MS_lab, 'B', 'C', 'BC', '106_Zn/Ni')

MS_TK01_PbZn = MS_TK01_Pb_df.join(MS_TK01_Zn_df, how='outer')
MS_TK01_CuNi = MS_TK01_Cu_df.join(MS_TK01_Ni_df, how='outer')
MS_TK01_Co = MS_TK01_Co_df
MS_TK01_FeMn = MS_TK01_Fe_df.join(MS_TK01_Mn_df, how='outer')
MS_TK01_CaCr = MS_TK01_Cr_df.join(MS_TK01_Ca_df, how='outer')
MS_TK01_AlSi = MS_TK01_Si_df.join(MS_TK01_Al_df, how='outer')
MS_TK01_pHMg = MS_TK01_Mg_df.join(MS_TK01_pH_df, how='outer')
MS_TK01_ZnNiORP = MS_TK01_ORP_df.join(MS_TK01_ZnNi_df, how='outer')
#--------------
MS_TK01_PbZnCuNi = MS_TK01_PbZn.join(MS_TK01_CuNi, how='outer')
MS_TK01_CoFeMnCr = MS_TK01_Co.join(MS_TK01_FeMn, how='outer')
MS_TK01_CaSiAlMg = MS_TK01_CaCr.join(MS_TK01_AlSi, how='outer')
MS_TK01_Fe2pHORP = MS_TK01_pHMg.join(MS_TK01_ZnNiORP, how='outer')

MS_TK01_PbZnCuNiCoFeMnCr = MS_TK01_PbZnCuNi.join(MS_TK01_CoFeMnCr, how='outer')
MS_TK01_CaSiAlMgFe2Fe3pHORP = MS_TK01_CaSiAlMg.join(MS_TK01_Fe2pHORP, how='outer')

MS_TK01_PbZnCuNiCoFeMnCrCaSiAlMgFe2Fe3pHORP = MS_TK01_PbZnCuNiCoFeMnCr.join(MS_TK01_CaSiAlMgFe2Fe3pHORP, how='outer')
df_main_106TK01 = MS_TK01_PbZnCuNiCoFeMnCrCaSiAlMgFe2Fe3pHORP
#--------------

In [63]:
# generates a dataframe for each column in the Analysis excel file 106VE04
MS_VE04_Ni_soln_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'D', '106VE04_Ni_soln')
MS_VE04_Pb_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'S', '106VE04_Pb')
MS_VE04_Zn_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'T', '106VE04_Zn')
MS_VE04_Cu_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'U', '106VE04_Cu')
MS_VE04_Ni_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'V', '106VE04_Ni')
MS_VE04_Co_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'W', '106VE04_Co')
MS_VE04_Fe_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'X', '106VE04_Fe')
MS_VE04_Mn_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'Y', '106VE04_Mn')
MS_VE04_Cr_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'Z', '106VE04_Cr')
MS_VE04_Ca_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'AA', '106VE04_Ca')
MS_VE04_Si_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'AB', '106VE04_Si')
MS_VE04_Al_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'AC', '106VE04_Al')
MS_VE04_Mg_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'AD', '106VE04_Mg')
MS_VE04_S_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'AE', '106VE04_S')
MS_VE04_pH_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'AG', '106VE04_pH')
MS_VE04_ORP_df = generate_df_from_analysis(MS_VE04_lab, 'B', 'C', 'AH', '106VE04_ORP')

In [ ]:
####-------------------Space for the 106TK01 and 106VE04 data to plot------------------------------------------
#####---------------------Pidaat and Ni recovery calc
df_Pi = pd.read_excel(r'C:\Users\v.t.flores\Documents\MS Flow1.xlsx', sheet_name='PiData_MS FLOW', index_col=False)
#Drop the row[1]
df_Pi = df_Pi.drop(df_Pi.index[0])
#Step2: Parse the DATETIME column
df_Pi['DATETIME'] = df_Pi['DATETIME'].apply(lambda x: parse(str(x)))
df_Pi.set_index('DATETIME', inplace=True)
df_comb = df_Pi
for i in df_comb.columns:
    df_comb[i] = pd.to_numeric(df_comb[i], errors='coerce')
df_comb['MS_Flow'] = np.where(df_comb['MS_FLOW_A']>df_comb['MS_FLOW_B'], df_comb['MS_FLOW_A'], df_comb['MS_FLOW_B'])
df_comb['Operation_Load'] = df_comb['MS_Flow']*100/1600
df_comb = df_comb[['MS_Flow', 'Operation_Load']]
dfn = df_comb['Operation_Load']
Ni = MS_TK01_Ni_df.join(MS_TK01_Fe_df, how= 'outer')
pHNiSoln = MS_TK01_pH_df.join(MS_VE04_Ni_soln_df, how= 'outer')
df_3 = Ni.join(pHNiSoln, how='outer')
df_4 = df_3.join(dfn, how='outer')
df_5 = df_4.interpolate(method='linear', limit_direction='forward', axis=0)
df_5 = df_5.dropna(how='any')
df_5['Ni_recovery'] = (df_5['106TK01_Ni']-df_5['106VE04_Ni_soln'])*100/df_5['106TK01_Ni']

In [123]:
#-------------------------------------------------------------------------------------------------------------------------#
#---------------------Code for predicting the 106VE04 Ni soln by applying the coefficients  from RidgeRegression----------#

#----Importing the Pi data for MS flow, rec gas, feed temp

df_MSPi = pd.read_excel(r'C:\Users\v.t.flores\Documents\MS Flow1.xlsx', sheet_name='PiData_MS FLOW', index_col=False)

#------------------Drop the row[0]. This is the title tag use in the pi datalink. We don't need it in pandas df.
#------------------What remain are the column title and the rest of the rows of data.

df_MSPi = df_MSPi.drop(df_MSPi.index[0])

#Step2: Parse the DATETIME column
df_MSPi['DATETIME'] = df_MSPi['DATETIME'].apply(lambda x: parse(str(x)))
df_MSPi.set_index('DATETIME', inplace=True)
df_MSPi = df_MSPi

# Step3: Convert each column to numeric to handle nan values and other comments like 'No data...'. Convering to numeric
#------ converts them into a NaN value that pandas understands.
for i in df_MSPi.columns:
    df_MSPi[i] = pd.to_numeric(df_MSPi[i], errors='coerce')

# Step4: Select which FT is online based on their flowrates. The one with the greater value is online.

df_MSPi['MS_Flow'] = np.where(df_MSPi['MS_FLOW_A']>df_MSPi['MS_FLOW_B'], df_MSPi['MS_FLOW_A'], df_MSPi['MS_FLOW_B'])


# Step5: Selec the three rows for MS flow, feed temp, rec gas. Then put them in a single dataframe.
df_MSPi_1 = df_MSPi[['MS_Flow', 'MS_FEED_TEMP', 'REC_GAS']]

#-------Get the columns for 106TK01 lab resulsts on MG, pH, Ni, Fe
df_VE04_Ni_pred = df_main_106TK01[['106TK01_Mg', '106TK01_pH', '106TK01_Ni', '106TK01_Fe']]
df_VE04Ni_MSpi = df_MSPi_1.join(df_VE04_Ni_pred, how='outer')
df_VE04Ni_MSpi = df_VE04Ni_MSpi.join(MS_VE04_Ni_soln_df, how='outer')

# Step6: Create a dictionary for the regression coefficients.
VE04_Ni_pred_coef_ = {0: 0.0000564586689, 1: -0.00439535054, 2: -0.0000177104562, 
                     3: -0.000172540353, 4:-0.00566003668, 5:0.0479078308, 6:-0.00113562602}
VE04_Ni_pred_intrcpt_ = 0.167293627122493

# Step7: Create columns for each of the coefficients.
df_VE04Ni_MSpi['Coef0'] = VE04_Ni_pred_coef_[0]
df_VE04Ni_MSpi['Coef1'] = VE04_Ni_pred_coef_[1]
df_VE04Ni_MSpi['Coef2'] = VE04_Ni_pred_coef_[2]
df_VE04Ni_MSpi['Coef3'] = VE04_Ni_pred_coef_[3]
df_VE04Ni_MSpi['Coef4'] = VE04_Ni_pred_coef_[4]
df_VE04Ni_MSpi['Coef5'] = VE04_Ni_pred_coef_[5]
df_VE04Ni_MSpi['Coef6'] = VE04_Ni_pred_coef_[6]

#Step8: Fill the empty cells using interpolate.
df_VE04Ni_MSpi = df_VE04Ni_MSpi.interpolate(method='linear', limit_direction='forward', axis=0)

# Step9-----Solving for the VE04 Ni concentration prediction using the coefficients from ridgeregression
    
df_VE04Ni_MSpi['VE04_Ni_soln_pred'] = df_VE04Ni_MSpi['MS_Flow']*df_VE04Ni_MSpi.Coef0 + \
                                      df_VE04Ni_MSpi['MS_FEED_TEMP']*df_VE04Ni_MSpi.Coef1 + \
                                      df_VE04Ni_MSpi['REC_GAS']*df_VE04Ni_MSpi.Coef2 + \
                                      df_VE04Ni_MSpi['106TK01_Mg']*df_VE04Ni_MSpi.Coef3 + \
                                      df_VE04Ni_MSpi['106TK01_pH']*df_VE04Ni_MSpi.Coef4 + \
                                      df_VE04Ni_MSpi['106TK01_Ni']*df_VE04Ni_MSpi.Coef5 + \
                                      df_VE04Ni_MSpi['106TK01_Fe']*df_VE04Ni_MSpi.Coef6 + \
                                      VE04_Ni_pred_intrcpt_
# Step10: Adjust the predicted values for retention time by temporarily transfering it to a separate df, 
#         then adding 2 hrs to it's datetime. Then rejoining it to the df_VE04Ni_MSpi df.
df_VE04_Ni_Pred_solo = df_VE04Ni_MSpi['VE04_Ni_soln_pred']

df_VE04Ni_MSpi = df_VE04Ni_MSpi.drop('VE04_Ni_soln_pred', axis=1) # Drop the VE04_Ni_soln_pred from the main df. Join it after adding retention time
df_VE04_Ni_Pred_solo = df_VE04_Ni_Pred_solo.reset_index()
df_VE04_Ni_Pred_solo['DATETIME'] =  df_VE04_Ni_Pred_solo['DATETIME'] + datetime.timedelta(hours=2)
df_VE04_Ni_Pred_solo.set_index('DATETIME', inplace=True)

# Join the time adjusted VE04_Ni_pred_solo
df_VE04Ni_MSpi = df_VE04Ni_MSpi.join(df_VE04_Ni_Pred_solo, how='outer')

print('VE04 PRED part Done', datetime.datetime.now())

#-------------------------------------------End of VE04 Ni Prediction Code -------------------------------------------####


